In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import math

D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1ty

In [2]:
RandomSeed = 9
np.random.seed(RandomSeed)
tf.set_random_seed(RandomSeed)

In [3]:
class PtPINNsss:
    # Initialize the class
    def __init__(self, x, t, u, lb, ub, lbp,ubp, layers,result_weights_values,result_biases_values):
        
        X = np.concatenate([x, t], 1)

        self.X = X
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        
        self.u = u      
        self.lb = lb
        self.ub = ub
        self.hsadasjd=0
        self.ubp = ubp
        self.lbp = lbp        
        # Initialize NNs
        self.layers = layers
        self.hh=[]
        self.hh1=[]      
        
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Calculate the values and convert them to float32
        value_float64 = 0.5
        self.b = tf.Variable(value_float64, dtype=tf.float64)
        
        value_float64=  1.0/(math.cosh(5*0.5)-1)       
        
        self.a = tf.constant(value_float64, dtype=tf.float64)     
        
        
        
        self.weights_values = result_weights_values        
        self.biases_values = result_biases_values    

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))    

        self.x_f_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.x_lb_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_b_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        
        self.x_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float64, shape=[None, self.u.shape[1]])

        self.u_pred, _ ,self.uuuuuu = self.net_AC(self.x_tf, self.t_tf)
        self.u_lb_pred, self.ux_lb_pred,_ = self.net_AC(self.x_lb_tf, self.t_b_tf)
        self.u_ub_pred, self.ux_ub_pred,_ = self.net_AC(self.x_ub_tf, self.t_b_tf)

        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        

        
        self.lossS = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))                                              
        self.lossB = tf.reduce_mean(tf.square(self.u_lb_pred - self.u_ub_pred)) + tf.reduce_mean(tf.square(self.ux_lb_pred - self.ux_ub_pred))
        self.lossfu =  tf.reduce_mean(tf.square(self.f_pred))    
        

        self.optimizer_Adam = tf.train.AdamOptimizer()

        self.loss  =   self.lossB + self.lossfu
        
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)      

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    
        init = tf.global_variables_initializer()
        self.sess.run(init)
        self.save = tf.train.Saver(max_to_keep=1)
              
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = tf.Variable(tf.zeros([layers[l], layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    
    def initialize_NN2(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases      
    
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float64))
    
    def custom_function(self,t):
        ab= self.b     
        condition2 = tf.less_equal(t, 0.4)
        condition3 = tf.less_equal(t, ab)
        condition4 = tf.less_equal(ab, 0.6) 
        BB=5*(t-0.4)
        B=1/(ab-0.4)*(t-0.4)
        
        return tf.where(condition2, tf.zeros_like(t), tf.where(condition4,tf.where(condition3, -2*B**3+3*B**2,tf.zeros_like(t)+1),-2*BB**3+3*BB**2))
    
 

    def input_encoding(self, t, x):    
        H=tf.concat([t,1+0*tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.sin(math.pi*x)],1) 
        H=tf.concat([H,tf.cos(2*math.pi*x)],1)
        H=tf.concat([H,tf.sin(2*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(3*math.pi*x)],1)
        H=tf.concat([H,tf.sin(3*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(4*math.pi*x)],1)
        H=tf.concat([H,tf.sin(4*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(5*math.pi*x)],1)
        H=tf.concat([H,tf.sin(5*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(6*math.pi*x)],1)
        H=tf.concat([H,tf.sin(6*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(7*math.pi*x)],1)
        H=tf.concat([H,tf.sin(7*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(8*math.pi*x)],1)
        H=tf.concat([H,tf.sin(8*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(9*math.pi*x)],1)
        H=tf.concat([H,tf.sin(9*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(10*math.pi*x)],1)
        H=tf.concat([H,tf.sin(10*math.pi*x)],1)           
        
        return H


    def neural_net(self, x,t, weights, biases,weights_values,biases_values):
        X = tf.concat([x,t],1)
        tt=5*t
        num_layers = len(weights) + 1
        H = self.input_encoding(tt, x)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            W1 = weights_values[l]
            b1 = biases_values[l]   
            H0 = tf.add(tf.matmul(H, W), b)
            H1 = tf.add(tf.matmul(H, W1), b1)
            
            B = self.custom_function(t)   
            H0 = H0*B              
            
            H  = tf.add(H0,H1)
            H =  tf.tanh(H)
        
        W = weights[-1]
        b = biases[-1]
        W1 = weights_values[-1]
        b1 = biases_values[-1]  
        
        H0 = tf.add(tf.matmul(H, W), b)
        
        B = self.custom_function(t)   
        H0 = H0*B            
        
        H1 = tf.add(tf.matmul(H, W1), b1) 
        H  = tf.add(H0,H1)      
        Y = H
        return Y
    
    def net_AC(self, x, t):
        u = self.neural_net(x,t, self.weights, self.biases,self.weights_values, self.biases_values)
        u_x = tf.gradients(u, x)[0]
        u_t = tf.gradients(u, t)[0]

        return u, u_x, u_t
          
    

    def net_f(self, x, t):
        u, u_x, u_t = self.net_AC(x, t)
        
        u_xx = tf.gradients(u_x, x)[0]
        
        f_u = u_t - 0.0001*u_xx+5*u**3-5*u
        
        return f_u
    
    def callback(self, loss, a,b,lossfu, lossS, lossB):
        sss=self.hsadasjd
        if sss%1000==0:
            print('Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e ' % (loss, lossfu, lossS, lossB))
        sss=sss+1
        self.hsadasjd=sss 
        self.hh.append(a)      
        self.hh1.append(b)        
    def train(self, nIter, Nf, Nn, Nb):

        X_train = self.lbp + (self.ubp-self.lbp)*lhs(2, Nf)
        self.xtrain_f = X_train[:,0:1]
        self.ttrain_f = X_train[:,1:2] 
        
        X_lb_train = self.lbp + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_lb = X_lb_train[:,0:1]
        self.ttrain_b = X_lb_train[:,1:2]
        
        X_ub_train = [self.ubp[0],self.lbp[1]] + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_ub = X_ub_train[:,0:1]
        
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u,
                   self.x_lb_tf: self.xtrain_lb, self.t_b_tf: self.ttrain_b, 
                   self.x_ub_tf: self.xtrain_ub, 
                   self.x_f_tf: self.xtrain_f, self.t_f_tf: self.ttrain_f}

        for it in range(nIter):
            loss_value = self.sess.run(self.loss, tf_dict)
            lossfu = self.sess.run(self.lossfu, tf_dict)
            lossS = self.sess.run(self.lossS, tf_dict)
            lossB = self.sess.run(self.lossB, tf_dict)
            self.sess.run(self.train_op_Adam, tf_dict)
                
            ab= self.sess.run(self.a)
            self.hh.append(ab)       
            abc= self.sess.run(self.b)
            self.hh1.append(abc)    

            print(abc)                    
            if it % 1000 == 0:
                print('It: %d, Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e' % (it, loss_value, lossfu, lossS, lossB))
                        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, method = 'L-BFGS-B', options = {'maxiter': 50000,'maxfun': 50000,'maxcor': 50,'maxls': 50,'ftol' : 1.0 * np.finfo(float).eps})                                                                                                         
        self.optimizer.minimize(self.sess, feed_dict = tf_dict, fetches = [self.loss,self.a,self.b,  self.lossfu, self.lossS, self.lossB], loss_callback = self.callback)        
                                    
    
    def predict(self, x, t):
        
        tf_dict = {self.x_tf: x, self.t_tf: t}
        u_star = self.sess.run(self.u_pred, tf_dict)
        u_starssss = self.sess.run(self.uuuuuu, tf_dict)        
        return u_star,u_starssss

    def saver(self, string):
        self.save.save(self.sess, 'ckpt/'+string)
        
        
    def sssss(self):
        return self.hh,  self.hh1           
        
        
    def restore(self):
        model_file = tf.train.latest_checkpoint('ckpt/')
        self.save.restore(self.sess, model_file)

In [4]:
import pickle

In [5]:
with open('bcweights.pkl', 'rb') as f:
    weights_values = pickle.load(f)

In [6]:
with open('bcweights1.pkl', 'rb') as f:
    biases_values = pickle.load(f)

In [7]:
with open('2bcweights.pkl', 'rb') as f:
    weights_values2 = pickle.load(f)
with open('2bcweights1.pkl', 'rb') as f:
    biases_values2 = pickle.load(f)

In [8]:
result_weights_values = [x + y for x, y in zip(weights_values, weights_values2)]
result_biases_values = [x + y for x, y in zip(biases_values, biases_values2)]

In [9]:
if __name__ == "__main__": 
           
    
    # Doman bounds
    lb = np.array([-1.0, 0])
    ub = np.array([1.0, 0.2])

    # Pre-training interval
    lbp = np.array([-1.0, 0.4])    
    ubp = np.array([1.0,  0.6])


    
    layers = [22, 100, 100, 100, 100, 1]
    
    data = scipy.io.loadmat('AC.mat')
    
    t = data['tt'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = data['uu']
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]

    def IC(x):
        u = x**2*np.cos(np.pi*x)
        return u

    N0 = 1200
    x=np.linspace(-1,1,N0).flatten()[:,None]  
    X0 =x
    T0 = np.full((N0,1), lb[1])
    U0 = IC(X0)
      
    model2 = PtPINNsss(X0, T0, U0, lb, ub,lbp, ubp, layers,result_weights_values,result_biases_values)                        
    model2.train(5000, 4000, 1200, 1200)  
    model2.saver('testmodel.ckpt')
    


    u_pred ,_= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0.5
It: 0, Loss: 3.408481e-02, Lossfu: 3.408e-02, LossS: 6.774e-03, LossB: 7.159e-29
0.500744134332429
0.5013457887791394
0.5019752615148113
0.5026953023250802
0.5034803383623208
0.5043071814823276
0.5051597570317445
0.506025078490113
0.5068843710778134
0.507717058122243
0.508508440800108
0.509258795868255
0.509983470272763
0.5107035825210233
0.5114311416143728
0.5121595831331054
0.5128711673714085
0.5135478824428082
0.5141814985937672
0.5147736107937351
0.515329252784908
0.5158524212082335
0.5163436554190107
0.5168010795941834
0.5172232352061131
0.5176111045948922
0.5179687693590617
0.5183016543332455
0.5186144037385406
0.5189095589939697
0.5191879867305407
0.5194498432483428
0.5196955685768734
0.5199264409473513
0.5201443189411388
0.5203509982124794
0.5205477993393636
0.5207353034076724
0.520913606101303
0.521082506136603
0.5212417234030053
0.5213912601691353
0.5215

0.5258078813525596
0.5258165108808794
0.5258251464071083
0.5258337877751403
0.5258424348384979
0.5258510874608868
0.5258597451521987
0.5258684175070655
0.5258771034932365
0.5258858021745358
0.5258945127016444
0.5259032343029288
0.5259119662783047
0.5259207079915191
0.5259294586745497
0.5259382178194844
0.5259469849635798
0.5259557596848738
0.5259645415981832
0.5259733306059756
0.5259821263595227
0.5259909285397362
0.5259997368543338
0.5260085510352811
0.5260173708364663
0.5260261960315797
0.526035026412183
0.526043862481648
0.5260527039952588
0.5260615507279591
0.5260704024724697
0.526079259038338
0.5260881280057168
0.5260970084331615
0.5261058994670214
0.5261148003328744
0.5261237103278283
0.5261326311392187
0.5261415619567499
0.5261505020448307
0.5261594507360173
0.5261684074243644
0.5261773715601422
0.5261863426429129
0.5261953214538535
0.5262043074582392
0.5262133001706236
0.5262222985987672
0.5262313023949976
0.5262403112419083
0.5262493248501104
0.5262583429540535
0.5262673653103

0.5300321746987993
0.5300423594411112
0.5300525426714789
0.5300627238589966
0.5300729025249831
0.5300830782373583
0.5300932505655377
0.5301034191624683
0.5301135837152352
0.5301237447927364
0.5301339020543697
0.5301440551927957
0.5301542039304765
0.5301643480172823
0.5301744872275536
0.5301846213574224
0.530194750223162
0.5302048736589356
0.5302149955828996
0.530225129322508
0.5302352730192345
0.5302454249980607
0.5302555837500355
0.5302657479150107
0.5302759169549314
0.5302860896956028
0.5302962650777643
0.5303064421459017
0.530316620037643
0.5303268007333717
0.5303369832566933
0.5303471667272109
0.5303573503510938
0.5303675333838648
0.5303777151834537
0.5303878951709612
0.530398072824506
0.5304082476731353
0.5304184192920033
0.5304285872981128
0.5304387513461875
0.5304489130272132
0.5304590718712909
0.5304692274546624
0.5304793793955463
0.5304895273498665
0.530499672528262
0.5305098144987183
0.5305199528717279
0.530530087296152
0.5305402174554243
0.5305503431717744
0.5305604641781276

0.5342326663974478
0.5342420692429677
0.5342514697942113
0.5342608675225419
0.5342702619519014
0.5342796526539124
0.5342890392426631
0.5342984224213881
0.5343078017720172
0.5343171769183434
0.5343265490806687
0.534335917799429
0.5343452826614431
0.534354643295128
0.5343639993661048
0.5343733505738377
0.5343826966480216
0.5343920387004688
0.534401376376035
0.5344107093553625
0.5344200373513799
0.5344293601059544
0.534438677387348
0.5344479889873912
0.5344572965799679
0.5344665998115826
0.5344758983642123
0.5344851919517455
0.5344944803169579
0.5345037733113837
0.5345130711686276
0.5345223726446667
0.5345316766188384
0.5345409820820489
0.5345502881259587
0.5345595939330652
0.5345688987674408
0.5345782019668109
0.53458750293487
0.5345968017381784
0.5346060978338159
0.5346153907331125
0.5346246799964188
0.5346339652283445
0.5346432460733705
0.5346525263207512
0.5346618052714968
0.5346710822961567
0.5346803568279074
0.5346896297270485
0.5346989003974103
0.5347081683021578
0.5347174329580328

0.5380135487598864
0.5380213201125893
0.5380290938257375
0.5380368689802126
0.5380446447484194
0.5380524203853511
0.5380601952203734
0.5380679686499034
0.5380757401307746
0.5380835091742959
0.5380912753408551
0.5380990382351706
0.5381067975017458
0.538114554004823
0.5381223073387016
0.5381300571385056
0.5381378030762075
0.538145544856978
0.5381532822158627
0.5381610149148226
0.5381687427400803
0.5381764654997114
0.5381841905218832
0.5381919168991366
0.5381996438143446
0.5382073705318692
0.5382150964405261
0.5382228209390229
0.5382305434863329
0.5382382635956156
0.5382459808288618
0.53825369479211
0.5382614051310778
0.5382691222413233
0.5382768447726223
0.5382845715085843
0.5382923013535543
0.538300033320755
0.5383077665215894
0.5383155074864834
0.5383232547729063
0.5383310070816796
0.5383387632429754
0.5383465222034355
0.538354283014432
0.5383620448217035
0.5383698068559473
0.5383775684246203
0.538385332497574
0.5383930981545558
0.5384008645673132
0.5384086309903108
0.5384163967523402


...

0.5492195208442203
0.5492242074399228
0.5492288924070989
It: 3000, Loss: 9.532337e-05, Lossfu: 9.532e-05, LossS: 6.774e-03, LossB: 3.146e-29
0.5492335752037485
0.5492382562909572
0.5492429348142055
0.5492476117154006
0.5492522907018322
0.5492569730364851
0.5492616557240272
0.5492663417310202
0.5492710260776641
0.5492757148971688
0.5492803994226308
0.5492850919677633
0.5492897765021713
0.5492944780808707
0.5492991687991948
0.5493039016213892
0.5493086306796009
0.5493134765333656
0.5493183814071556
0.5493236518261817
0.5493293004279359
0.5493362223337831
0.5493449323546413
0.5493584124239586
0.5493793478260492
0.5494181633471505
0.5494828647246197
0.5496007156966646
0.5497709748449104
0.5499889336226068
0.550223200348132
0.550434586373037
0.5506294127400149
0.5508455073766748
0.5510580559103361
0.551262992075979
0.5514572889165582
0.5516483862520208
0.5518388120109576
0.552012121338687
0.5521713952670569
0.5523271459180291
0.5524751853030964
0.5526096314743253
0.552736333083182
0.5528584

0.5582451984026594
0.5582504891940555
0.5582557833419148
0.5582610802129301
0.558266381812976
0.5582716873478719
0.5582769961109033
0.5582823074456174
0.5582876207947239
0.558292935604433
0.5582982514446809
0.5583035678627211
0.5583088844965232
0.5583142010019627
0.5583195197444782
0.5583248402172003
0.5583301618755053
0.5583354843155566
0.5583408071078233
0.5583461298931551
0.5583514523684787
0.5583567742017307
0.55836209517459
0.5583674150529064
0.5583727336433747
0.5583780507467743
0.558383366170393
0.5583886797857374
0.5583939914174213
0.5583993009667901
0.5584046082999076
0.5584099133180077
0.558415215936044
0.5584205160534823
0.558425813615478
0.5584311085373087
0.5584364007721456
0.5584416902590795
0.5584469769488812
0.5584522605523343
0.5584575410493712
0.5584628184346838
0.5584680926908528
0.5584733638124972
0.5584786317873528
0.5584838966115795
0.5584891582756087
0.5584944167758459
0.5584996721063449
0.5585049242382684
0.5585101731731734
0.5585154189056162
0.5585206614412841


0.5661749336011213
0.5661801424920062
0.5661853582554691
0.5661905699448284
0.5661957829389284
0.5662009954796491
0.5662062044119129
0.5662114159576165
0.566216622805578
0.5662218315012033
0.5662270372410358
0.5662322413267035
0.5662374453424422
0.566242645706917
0.5662478473107414
0.5662530457664051
0.5662582434249822
0.56626343967084
0.5662686322206493
0.5662738251149674
0.5662790132396136
0.5662842013420285
0.566289385450144
0.5662945673386339
0.5662997468964381
0.566304922234011
0.5663100965179847
0.5663152660890023
0.5663204346842311
0.5663255992565047
0.5663307617030168
0.5663359212755538
0.5663410779084643
0.5663462327108472
0.5663513843278503
0.5663565340479894
0.5663616806788534
0.5663668247370598
0.566371966513068
0.5663771051098794
0.566382241907718
0.5663873750443806
0.5663925063304482
0.5663976342398971
0.5664027603089279
0.5664078839677075
0.5664130051155716
0.5664181243539939
0.5664232407702453
0.5664283554170539
0.5664334677055991
0.5664385783743328
0.5664436869732212
0

0.5753790606534849
0.5753844019995414
0.5753897408947034
0.5753950773253914
0.5754004112810764
0.5754057427514733
0.5754110717286821
0.5754163982052126
0.5754217221751597
0.5754270436334504
0.5754323625759373
0.575437678999534
0.5754429929014676
0.5754483042801615
0.5754536129812264
0.5754589190195305
0.5754642224085788
0.5754695231611413
0.5754748212889914
0.5754801168033676
0.5754854097145335
0.575490700032392
0.5754959877659458
0.5755012727934953
0.5755065551361341
0.5755118348137758
0.5755171118442156
0.5755223862443218
0.5755276580293714
0.5755329272138238
0.5755381940515422
0.5755434585313909
0.5755487206437219
0.5755539803802079
0.5755592378109887
0.5755644929227623
0.5755697457039847
0.5755749961446898
0.5755802442363492
0.5755854899717913
0.5755907333448256
0.5755959743504054
0.5756012129841613
0.5756064492428006
0.5756116831235114
0.5756169146243749
0.5756221437438199
0.5756273704809469
0.5756325948351121
0.5756378168062186
0.5756430363943689
0.575648253600176
0.5756534697751

0.5942245225277816
0.5942277972220719
0.5942310724959501
0.5942343480171058
0.5942376239828777
0.5942408999715677
0.5942441763292282
0.594247452486242
0.5942507290086142
0.5942540050798014
0.5942572815997711
0.5942605573589416
0.5942638337745718
0.594267109002008
0.5942703852819284
0.5942736597390931
0.594276935946218
0.5942802093384527
0.5942834856732453
0.5942867575941914
0.5942900344885957
0.5942933043320335
0.5942965826283201
0.5942998494676311
0.5943031336292028
0.5943064014983725
0.5943096967834898
0.5943129628133994
0.5943162750585961
0.5943195369121845
0.594322880642425
0.5943261404481465
0.5943295526394852
0.5943328325320751
0.5943364135092541
0.5943398119749395
0.5943438534553328
0.594347744668915
0.5943531304001498
0.5943588172561092
0.5943683977124757
0.5943802729651491
0.5944029529296047
0.5944338965068002
0.5944927415238961
0.5945670858508512
0.5946887279479333
0.5948089676986481
0.5949479231606841
0.5950776343853577
0.5951943915113368
0.5953409078214874
0.595473539584302

In [10]:
    t = data['tt'].flatten()[:,None][0:41] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,0:41] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.60)
Error2 u: 1.119412e-01
Error1 u: 4.014867e-02
Errorf u: 1.635085e-01


In [22]:
    t = data['tt'].flatten()[:,None][80:121] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,80:121] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.60)
Error2 u: 3.834297e-04
Error1 u: 1.702582e-04
Errorf u: 1.585928e-03


In [23]:
    scipy.io.savemat("solution3.mat", {'u': u_pred})

In [24]:
    t = data['tt'].flatten()[:,None][81:121] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,81:121] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.60)
Error2 u: 3.851745e-04
Error1 u: 1.713762e-04
Errorf u: 1.585928e-03


In [25]:
    scipy.io.savemat("solution33.mat", {'u': u_pred})

In [18]:
weights_values2 = model2.sess.run(model2.weights)
biases_values2 = model2.sess.run(model2.biases)

In [19]:
import pickle
with open('3bcweights.pkl', 'wb') as f:
    pickle.dump(weights_values2, f)
with open('3bcweights1.pkl', 'wb') as f:
    pickle.dump(biases_values2, f)